In [1]:
from logic.getting import *

In [4]:
from logic.scaler import *

In [5]:
from logic.supporting_functions import *

In [6]:
from logic.final_preproc_function import *

In [7]:
from logic.windows_preproc import *

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [9]:
from tensorflow.keras.layers import Normalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten, LSTM, GRU, Embedding
import tensorflow.keras.layers as layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import hinge, Recall, Precision, PrecisionAtRecall
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.utils as utils
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [10]:
price = get_price_data_raw('AAPL')

/Users/oscarthieleserrano/.pyenv/versions/3.10.6/envs/portfolio_manager/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [11]:
price = price.sort_values(by='datetime')

In [12]:
tech = get_technical_analysis('AAPL')

In [13]:
macro = get_macro_data()

In [14]:
tech = tech.sort_values(by='timestamp_field_0')

In [15]:
macro = macro.sort_values(by='timestamp')

In [16]:
price_re = price[::2]

In [17]:
tech_re = tech[::2]

In [18]:
price_re.shape

(69685, 6)

In [19]:
macro_re = macro[::2]

In [21]:
X, y, log_df = create_x_y(price, tech)

In [22]:
price.shape

(139370, 6)

In [23]:
X.shape

(139309, 24)

In [24]:
X_train = X[:100000]

In [25]:
X_val = X[100000:110000]

In [26]:
X_test = X[110000:]

In [27]:
y_train = y[:100000]

In [28]:
y_val = y[100000:110000]

In [29]:
y_test = y[110000:]

In [30]:
scaler = initialize_scaler(X_train)

In [31]:
X_train_sc = transform_scaler(scaler, X_train)

In [32]:
X_val_sc = transform_scaler(scaler, X_val)

In [33]:
X_test_sc = transform_scaler(scaler, X_test)

In [34]:
X_tr_w, y_tr_w = create_lstm_windows(X_train_sc, y_train, 20)

In [35]:
X_val_w, y_val_w = create_lstm_windows(X_val_sc, y_val, 20)

In [36]:
X_te_w, y_te_w = create_lstm_windows(X_test_sc, y_test, 20)

In [37]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Conv1D, BatchNormalization, GlobalAveragePooling1D, Dense, Dropout, Input, Concatenate
from tensorflow.keras.models import Model


In [59]:
def build_lstm_fcn(input_shape):
    # Input layer
    input_layer = Input(shape=input_shape)

    # LSTM Block
    lstm_layer = LSTM(units=128)(input_layer)

    # FCN Block
    conv1 = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu')(input_layer)
    conv1 = BatchNormalization()(conv1)
    conv2 = Conv1D(filters=256, kernel_size=5, padding='same', activation='relu')(conv1)
    conv2 = BatchNormalization()(conv2)
    conv3 = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(conv2)
    conv3 = BatchNormalization()(conv3)
    gap_layer = GlobalAveragePooling1D()(conv3)

    # Concatenate LSTM and FCN outputs
    concat_layer = Concatenate()([lstm_layer, gap_layer])

    # Output layer
    output_layer = Dense(1, activation='sigmoid')(concat_layer)

    # Define model
    model = Model(inputs=input_layer, outputs=output_layer)

    # Compile model
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [72]:
model = build_lstm_fcn(input_shape=(20,1))

In [73]:
es = EarlyStopping(patience=5, restore_best_weights=True)

In [74]:
X_tr_w[:,:,0:1].shape

(99980, 20, 1)

In [77]:
X_tr_w.shape

(99980, 20, 24)

In [78]:
model.fit(X_tr_w[:,:,0:1], y_tr_w, epochs=50, callbacks=[es], validation_data=(X_val_w, y_val_w))

Epoch 1/50
3121/3125 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5122 - loss: 0.6941

ValueError: Exception encountered when calling Functional.call().

[1mInput 0 of layer "conv1d_18" is incompatible with the layer: expected axis -1 of input shape to have value 1, but received input with shape (None, 20, 24)[0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(None, 20, 24), dtype=float32)
  • training=False
  • mask=None

In [171]:
model.evaluate(X_te_w, y_te_w)

916/916 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5038 - loss: 0.7068


[0.7177866101264954, 0.4996073544025421]

In [151]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Conv1D, BatchNormalization, GlobalAveragePooling1D, Dense, Dropout, Input, Concatenate, Bidirectional, Add, Activation, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

In [152]:
def attention_block(inputs):
    """Applies attention mechanism on the input tensor."""
    attention = Dense(1, activation='tanh')(inputs)
    attention = tf.keras.layers.Flatten()(attention)
    attention = tf.keras.layers.Activation('softmax')(attention)
    attention = tf.keras.layers.RepeatVector(inputs.shape[-1])(attention)
    attention = tf.keras.layers.Permute([2, 1])(attention)
    output = tf.keras.layers.Multiply()([inputs, attention])
    return output

In [153]:
def residual_block(inputs, filters, kernel_size):
    """Residual block with Conv1D and BatchNormalization."""
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same', activation='relu', kernel_regularizer=l2(0.001))(inputs)
    x = BatchNormalization()(x)
    x = Conv1D(filters=filters, kernel_size=kernel_size, padding='same', activation='relu', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    
    # Adjust input dimensions to match the output if necessary
    shortcut = Conv1D(filters=filters, kernel_size=1, padding='same')(inputs)
    shortcut = BatchNormalization()(shortcut)
    
    x = Add()([shortcut, x])
    x = Activation('relu')(x)
    return x


In [154]:
def build_enhanced_lstm_fcn(input_shape):
    # Input layer
    input_layer = Input(shape=input_shape)

    # Bidirectional LSTM Block
    lstm_layer = Bidirectional(LSTM(units=128, return_sequences=True))(input_layer)
    lstm_layer = Dropout(0.3)(lstm_layer)
    lstm_layer = Bidirectional(LSTM(units=64))(lstm_layer)
    lstm_layer = Dropout(0.3)(lstm_layer)

    # Attention mechanism applied to LSTM output
    lstm_layer = attention_block(lstm_layer)
    lstm_layer = Flatten()(lstm_layer)  # Flatten the output to 2D for concatenation

    # FCN Block with residual connections
    conv1 = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', kernel_regularizer=l2(0.001))(input_layer)
    conv1 = BatchNormalization()(conv1)
    conv2 = residual_block(conv1, filters=256, kernel_size=5)  # 1x1 conv will adjust filters
    conv3 = residual_block(conv2, filters=128, kernel_size=3)
    gap_layer = GlobalAveragePooling1D()(conv3)

    # Concatenate LSTM and FCN outputs
    concat_layer = Concatenate()([lstm_layer, gap_layer])

    # Dense layers with Dropout and Regularization
    dense_layer = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(concat_layer)
    dense_layer = Dropout(0.3)(dense_layer)
    dense_layer = Dense(64, activation='relu', kernel_regularizer=l2(0.001))(dense_layer)
    dense_layer = Dropout(0.3)(dense_layer)

    # Output layer
    output_layer = Dense(1, activation='sigmoid')(dense_layer)

    # Define model
    model = Model(inputs=input_layer, outputs=output_layer)

    # Compile model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [158]:
model = build_enhanced_lstm_fcn(input_shape=(20,27))

In [159]:
es = EarlyStopping(patience=5, restore_best_weights=True)

In [162]:
model.fit(X_tr_w, y_tr_w, epochs=50, callbacks=[es], validation_data=(X_val_w, y_val_w))

Epoch 1/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 153s 49ms/step - accuracy: 0.5135 - loss: 0.8361 - val_accuracy: 0.5003 - val_loss: 0.6943
Epoch 2/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 158s 50ms/step - accuracy: 0.5153 - loss: 0.6930 - val_accuracy: 0.5003 - val_loss: 0.6939
Epoch 3/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 159s 51ms/step - accuracy: 0.5154 - loss: 0.6927 - val_accuracy: 0.5003 - val_loss: 0.6947
Epoch 4/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 159s 51ms/step - accuracy: 0.5183 - loss: 0.6925 - val_accuracy: 0.5003 - val_loss: 0.6939
Epoch 5/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 164s 52ms/step - accuracy: 0.5162 - loss: 0.6927 - val_accuracy: 0.5003 - val_loss: 0.6941
Epoch 6/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 169s 54ms/step - accuracy: 0.5170 - loss: 0.6926 - val_accuracy: 0.5003 - val_loss: 0.6941
Epoch 7/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 170s 55ms/step - accuracy: 0.5180 - loss: 0.6925 - val_accuracy: 0.5003 - val_loss: 0.6936
Epoch 8/50
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 170s 55ms/step - accuracy: 

KeyboardInterrupt: 

In [163]:
model.evaluate(X_te_w, y_te_w)

916/916 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.4983 - loss: 0.6943


[0.6940942406654358, 0.5002902150154114]